# Autoencoder und CNN gleichzeitig trainieren

**Disclamer:** Wir wollten den Autoencoder und das CNN gleichzeitig trainieren um das Ergebnis zu verbessern. Leider haben wir irgendwo einen Bug im Code, welchen wir aber zeitlich nicht mehr lösen konnten und haben den Ansatz dann verworfen und uns auf andere Dinge fokusiert. Wir wollten jedoch trotzdem das Notebook abgeben, da trotzdem ein wenig Zeit reingeflossen ist.

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from keras import backend as K 
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras import regularizers
import tensorflow_model_optimization as tfmot
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import json
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.optimizers.legacy import SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay

In [20]:
data = pd.read_pickle('./data/images_df_numerical.pkl')
classes = data["Species"].unique()
number_of_classes = classes.size
X, y = data['data'], data['Species']
# Wir hatten massive Probleme mit der Begrenztheit unseres RAMs, weshalb wir versucht haben die Usage an mehreren Stellen zu reduzieren
X, y = np.stack(X).astype(np.float16), y.to_numpy().astype(np.uint8)


In [21]:
image_size = X[0].size
samples = X.size
with open("./data/meta.json","r") as file:  # Shape der Bilder nach dem resizen aus dem data_prep notebook
    image_meta = json.load(file) 
image_shape = (image_meta['h'],image_meta['w'],image_meta['c'])

In [22]:
X = X.reshape((-1,) + image_shape)
print(f"Image has shape: {image_shape}")

Image has shape: (82, 128, 3)


In [23]:
kfold = StratifiedKFold(n_splits=2, shuffle=True)

In [24]:
def train_val_test_split(train_indezes, test_indezes, tensor=False):
    X_train, X_val, y_train, y_val = train_test_split(X[train_indezes], y[train_indezes], test_size=0.2,stratify=y[train_indezes], random_state=42)
    X_test, y_test = X[test_indezes], y[test_indezes]

    
    # Das speichern als Tensor spart RAM
    if tensor:
        return (tf.convert_to_tensor(X_train),
                tf.convert_to_tensor(y_train), 
                tf.convert_to_tensor(X_val),
                tf.convert_to_tensor(y_val),
                tf.convert_to_tensor(X_test),
                tf.convert_to_tensor(y_test))
    else:
        return (X_train,y_train,X_val,y_val,X_test,y_test)

In [25]:
print(image_shape)

(82, 128, 3)


In [26]:
early_stopping = EarlyStopping(monitor='val_cnn_accuracy', patience=30, min_delta=0.001, start_from_epoch=15, restore_best_weights=True)
epochs = 200
batch_size = 32

filter_sizes = [32,32,32,32,32,1]

def create_autoencoder_model(silent=True):
    input = tf.keras.layers.Input(shape=image_shape, name="ae_input")
    # Encoder
    x = tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(input)
    x = tf.keras.layers.MaxPooling2D((2, 2), padding="same")(x)
    x = tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)
    x = tf.keras.layers.MaxPooling2D((2, 2), padding="same")(x)
    
    # Decoder
    x = tf.keras.layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same")(x)
    x = tf.keras.layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same")(x)
    x = tf.keras.layers.Conv2D(3, (3, 3), activation="sigmoid", padding="same")(x)
    x = tf.keras.layers.Cropping2D(((1,1),(0,0)))(x)
    
    # Autoencoder
    autoencoder = tf.keras.Model(input, x, name="autoencoder")
    return autoencoder


In [27]:
# Stoppt wenn bei 30 Epochen die Validation-Accuracy nicht um mehr als 0.001 besser wird
early_stopping = EarlyStopping(monitor='val_cnn_accuracy', patience=30, min_delta=0.01, start_from_epoch=15, restore_best_weights=True)
epochs = 200
batch_size = 32
dropout_rate = 0.4
weight_decay_alpha = 0.01

def create_cnn_model():
    model = tf.keras.models.Sequential(name="cnn")
    model.add(tf.keras.layers.Input(shape=image_shape, name="cnn_input"))
    model.add(tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay_alpha)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay_alpha)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay_alpha)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(weight_decay_alpha)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(weight_decay_alpha)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(number_of_classes, activation='softmax'))

    return model

In [28]:
def create_combined_model(autoencoder_model, cnn_model):
    # Eingabe für beide Modelle
    autoencoder_input = tf.keras.layers.Input(shape=image_shape, name="main_input")


    # Autoencoder-Modell aufrufen und Ausgabe erhalten
    autoencoder_output = autoencoder_model(autoencoder_input)

    # CNN-Modell aufrufen und Ausgabe erhalten
    cnn_output = cnn_model(autoencoder_output)
    
    # Gemeinsames Modell erstellen
    combined_model = tf.keras.Model(inputs=[autoencoder_input], outputs=[autoencoder_output, cnn_output], name="combined_model")

    return combined_model

In [29]:
def fit_model(model, X_train, y_train, X_val=None, y_val=None):
    # Das Model macht selbst den Validation Split
    if X_val is None or y_val is None:
        history = model.fit(
            X_train,
            y_train,
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[early_stopping, tfmot.sparsity.keras.UpdatePruningStep()],
            validation_split=0.2,
            verbose=1)
    # Wir übergeben dem Model Validation Daten
    elif X_val is not None and y_val is not None:
        history = model.fit(
            X_train,
            y_train,
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[early_stopping, tfmot.sparsity.keras.UpdatePruningStep()],
            validation_data=(X_val,y_val),
            verbose=1)
    return history

In [30]:
cnn_model = create_cnn_model()
autoencoder_model = create_autoencoder_model(False)
combined_model = create_combined_model(autoencoder_model, cnn_model)

In [31]:
def add_salt_and_pepper_noise(image, noise_ratio=0.2):
    noise_iamge = image.copy()
    h, w, c = noise_iamge.shape
    noisy_pixels = int(h * w * noise_ratio)
 
    for _ in range(noisy_pixels):
        row, col = np.random.randint(0, h), np.random.randint(0, w)
        if np.random.rand() < 0.5:
            noise_iamge[row, col] = [0, 0, 0] 
        else:
            noise_iamge[row, col] = [255, 255, 255]
 
    return (noise_iamge).astype(np.float16)

In [32]:
# Abspeichern der Erebnisse jedes Splits für die Confusion-Matrix
true_labels = list()
pred_labels = list()
train_accuracies = list()
test_accuracies = list()
val_accuracies = list()
train_losses = list()
val_losses = list()
test_losses = list()



for train_indezes, test_indezes in kfold.split(X, y):
    # wir löschen das Model der letzten Iteration aus dem Cache um RAM zu sparen
    K.clear_session()

    # Daten splitten
    X_train, y_train, X_val, y_val, X_test, y_test = train_val_test_split(train_indezes, test_indezes)

    X_train_noise = np.asarray([add_salt_and_pepper_noise(image) for image in X_train])

    print(X_train.shape)
    print(y_train.shape)
    print(X_train_noise.shape)
    print(X_test.shape)
    print(y_test.shape)
    print(X_val.shape)
    print(y_val.shape)
    
    # Resample nur Trainings- und Validationmenge
    #X_train, y_train = resample_after_split(X_train, y_train)
    #X_val, y_val = resample_after_split(X_val, y_val)

    # Das Model wird hier compiled, damit wir nach jeder Iteration das alte Model aus dem Cache löschen können  Adam(0.001)
    combined_model.compile(optimizer=Adam(learning_rate=0.001), loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], metrics=['binary_crossentropy', 'accuracy'])
    print(combined_model.summary())
    history = combined_model.fit({"main_input":X_train_noise}, [X_train, y_train], epochs=epochs, batch_size=batch_size, shuffle=True, validation_data=(X_val, [X_val, y_val]), callbacks=[early_stopping])
    #history = fit_model(combined_model, X_train, y_train, X_val, y_val)

    # Für die Confusion Matrix
    pred = combined_model.predict({"main_input":X_test})
    print(pred)
    #print(np.asarray(pred).shape)
    predictions = np.argmax(pred[1], axis=-1)
    true_labels.extend(y_test)
    pred_labels.extend(predictions)

    # Für die Accuracy-Curves
    train_accuracies.extend(history.history['cnn_accuracy'])
    val_accuracies.extend(history.history['val_cnn_accuracy'])

    # Für die Loss-Curves
    train_losses.extend(history.history['cnn_loss'])
    val_losses.extend(history.history['val_cnn_loss'])

    a = combined_model.evaluate({"main_input":X_test}, [X_test,y_test])
    print(a)
    test_accuracies.append(accuracy)
    test_losses.append(loss)



(1488, 82, 128, 3)
(1488,)
(1488, 82, 128, 3)
(1861, 82, 128, 3)
(1861,)
(373, 82, 128, 3)
(373,)
Model: "combined_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 main_input (InputLayer)     [(None, 82, 128, 3)]      0         
                                                                 
 autoencoder (Functional)    (None, 82, 128, 3)        29507     
                                                                 
 cnn (Sequential)            (None, 6)                 1441222   
                                                                 
Total params: 1470729 (5.61 MB)
Trainable params: 1469065 (5.60 MB)
Non-trainable params: 1664 (6.50 KB)
_________________________________________________________________
None
Epoch 1/200
 3/47 [>.............................] - ETA: 8s - loss: 12.8229 - autoencoder_loss: 2.1957 - cnn_loss: 2.7412 - autoencoder_mse: 0.3099 - autoencoder_accuracy: 0

Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x169969580>
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.11/3.11.6_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/weakref.py", line 369, in remove
    def remove(k, selfref=ref(self)):

KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
confusion_matrix = confusion_matrix(true_labels, pred_labels)
fig, ax = plt.subplots(figsize=(8, 6))
cm_display = ConfusionMatrixDisplay(confusion_matrix, display_labels=classes)
cm_display.plot(ax=ax, cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
epochs = len(train_losses)
plt.figure(figsize=(12, 4))

# Plotting Loss curves
plt.subplot(1, 2, 1)
plt.plot(range(1, epochs + 1), train_losses, label='Training Loss')
plt.plot(range(1, epochs + 1), val_losses, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plotting Accuracy curves
plt.subplot(1, 2, 2)
plt.plot(range(1, epochs + 1), train_accuracies, label='Training Accuracy')
plt.plot(range(1, epochs + 1), val_accuracies, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.savefig("./cnn_files/loss_and_accuraccy.png")
plt.show()

In [ ]:
print("Unser Modell:")
print(f"Avg. Val Accuracy: {sum(val_accuracies) / len(val_accuracies)}")
print(f"Best Val Accuracy: {max(val_accuracies)}")
print(f"Avg. Test Accuracy: {sum(test_accuracies) / len(test_accuracies)}")
print(f"Best Test Accuracy: {max(test_accuracies)}")
print(f"Avg. Test Loss: {sum(test_losses) / len(test_losses)}")